# SmolAgents Rehber

- Setup -> huggingface-cli login
- Basit Search 
- Smol Agents Yapısı
- Tools
- Agent Memory
- Model Ekleme (LiteLLM, Ollama, Gemini)
- Multi agent system
- AgenticRAG
- Secure Code Execution



# Basit Search

In [36]:
from smolagents import CodeAgent, WebSearchTool, InferenceClientModel, ToolCallingAgent, HfApiModel, DuckDuckGoSearchTool, LiteLLMModel
import os


In [ ]:
model = InferenceClientModel()
agent = CodeAgent(tools=[WebSearchTool()], model=model)
#agent = CodeAgent(tools=[WebSearchTool()], model=model, planning_interval=3) 

agent.run("How long it takes to a cheetah to run from TBMM (In Turkey) to MEF University")

# Model Ekleme (LiteLLM, Ollama, Gemini)

In [ ]:
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

In [ ]:
model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

In [ ]:
model=LiteLLMModel(
    model_id="ollama/gemma3:latest",
    api_base="http://localhost:11434",  # Adjust if using a remote server
    api_key="ollama"  # Replace with your API key if required
)

In [ ]:
agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=model)

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

In [ ]:

answer = agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")
print(answer)

# Multi Agent System

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool, LiteLLMModel, tool

from typing import Optional
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the model with Gemini API
model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)
@tool
def get_coordinates(location: str) -> dict:
    """
    Retrieve latitude and longitude for a specified location using Nominatim API.
    
    Args:
        location (str): The location to fetch coordinates for.
    
    Returns:
        dict: A dictionary containing latitude and longitude, or an error message if unsuccessful.
    """
    try:
        url = f"https://nominatim.openstreetmap.org/search?format=json&q={location}&limit=1"
        headers = {'User-Agent': 'SmolAgentsWeatherApp/1.0'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data and len(data) > 0:
                return {"latitude": float(data[0]["lat"]), "longitude": float(data[0]["lon"])}
            return {"error": f"No coordinates found for {location}"}
        return {"error": f"API error: {response.status_code}"}
    except Exception as e:
        return {"error": f"Error fetching coordinates for {location}: {str(e)}"}

@tool
def get_weather(location: str, celsius: Optional[bool] = True) -> str:
    """
    Fetch current weather information for a given location using Open-Meteo API.
    
    Args:
        location (str): The location to get weather information for.
        celsius (bool, optional): Use Celsius for temperature if True, Fahrenheit if False. Defaults to True.
    
    Returns:
        str: A formatted string with weather details or an error message with fallback dummy data.
    """
    try:
        coords = get_coordinates(location)
        if "error" in coords:
            unit = '°C' if celsius else '°F'
            temp = '7°C' if celsius else '45°F'
            return f"Could not retrieve coordinates for {location}: {coords['error']}. Falling back to dummy data: The weather in {location} is sunny with temperatures around {temp}."

        latitude = coords["latitude"]
        longitude = coords["longitude"]

        url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,relative_humidity_2m,rain,weather_code"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'current' in data:
                temp = data['current']['temperature_2m']
                humidity = data['current']['relative_humidity_2m']
                rain = data['current']['rain']
                weather_code = data['current']['weather_code']
                
                weather_desc = {
                    0: "clear sky", 1: "mainly clear", 2: "partly cloudy", 3: "overcast",
                    45: "fog", 51: "light drizzle", 61: "light rain", 63: "moderate rain",
                    65: "heavy rain", 71: "light snow", 73: "moderate snow", 75: "heavy snow",
                    80: "rain showers", 95: "thunderstorm"
                }.get(weather_code, "unknown weather condition")
                
                unit = "°C" if celsius else "°F"
                if not celsius:
                    temp = (temp * 9/5) + 32
                
                return f"The weather in {location} is {weather_desc} with a temperature of {temp:.1f}{unit}, humidity at {humidity}%, and rain at {rain}mm."
            return f"Could not retrieve weather data for {location}. No current data available."
        return f"Could not retrieve weather data for {location}. API error: {response.status_code}"
    except Exception as e:
        unit = '°C' if celsius else '°F'
        temp = '7°C' if celsius else '45°F'
        return f"Error fetching weather data for {location}: {str(e)}. Falling back to dummy data: The weather in {location} is sunny with temperatures around {temp}."

# Initialize the agent with weather and coordinates tools
web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()], 
    model=model,
    name="web_agent",
    description="Runs web searches. Give it your query as an argument.",
    )
weather_agent = ToolCallingAgent(
    tools=[get_weather, get_coordinates],
    model=model,
    name="weather_agent",
    description="Gets weather information for a given location. Give it the location as an argument.",
    )

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[weather_agent, web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)
# Run the agent to compare temperatures in different locations
#answer = web_agent.run("What is hotter right now? Istanbul or Brazil and also right now which country is the hottest?")
answer = manager_agent.run("What is hotter right now? Istanbul or Brazil and also right now which country is the hottest?")
print(answer)

# MEMORY

In [ ]:
from smolagents import ActionStep

system_prompt_step = agent.memory.system_prompt
print("The system prompt given to the agent was:")
print(system_prompt_step.system_prompt)

task_step = agent.memory.steps[0]
print("\n\nThe first task step was:")
print(task_step.task)

for step in agent.memory.steps:
    if isinstance(step, ActionStep):
        if step.error is not None:
            print(f"\nStep {step.step_number} got this error:\n{step.error}\n")
        else:
            print(f"\nStep {step.step_number} got these observations:\n{step.observations}\n")

# Agentic RAG

In [40]:
!pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade -q


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [41]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

Filter: 100%|██████████| 2647/2647 [00:00<00:00, 84850.95 examples/s]


In [42]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool(docs_processed)

In [43]:
agent = CodeAgent(
    tools=[retriever_tool], model=model, max_steps=4, verbosity_level=2
)

In [ ]:
agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)